In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Conv2D, MaxPooling2D, Dense


In [ ]:
image_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_gen.flow_from_directory("./train_data")

In [ ]:
### Custom model using CNN ###

model = Sequential()

input_shape = (150, 150, 3)

# add convolution layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=input_shape, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=input_shape, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=input_shape, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add flatten
model.add(Flatten())

# add dense layers
model.add(Dense(128, activation="sigmoid"))

# add drop out layer to help reduce over fitting by randomly turning neurons off during training
# model.add(Dropout(0.5))

# output layer
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# model summary
model.summary()

batch_size = 16
train_img_gen = image_gen.flow_from_directory("./train_data", target_size=input_shape[:2], batch_size=batch_size)

# class indices
print(train_img_gen.class_indices)

# fit the model
results = model.fit(train_img_gen, epochs=80, steps_per_epoch=600)


In [ ]:
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten

### VGG16 model ### 

# Set the image size and number of classes
img_width, img_height = 224, 224
num_classes = 10

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(img_width, img_height, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Build a new model on top of the pre-trained VGG16 model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up the data generators for training and validation
train_data_dir = './train_data'
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# val_datagen = ImageDataGenerator(rescale=1.0/255.0)
# val_generator = val_datagen.flow_from_directory(
#     val_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical')

# Train the model
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs)
    # validation_data=val_generator,
    # validation_steps=val_generator.n // batch_size)

# Save the trained model
model.save('image_classifier.h5')

In [ ]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

### ResNet50 model ### 

image_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data_dir = './train_data'
image_gen.flow_from_directory(train_data_dir)


# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model and add the base ResNet50 model and additional layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

train_img_gen = image_gen.flow_from_directory("./train_data", target_size=input_shape[:2], batch_size=64)
# Train the model
model.fit(train_img_gen, batch_size=64, epochs=10, )

# Evaluate the model
# score = model.evaluate(x_test, y_test, verbose=0)
